<a href="https://colab.research.google.com/github/shubhasreeav/SAM-for-WSSS/blob/main/SAM_FLOODNET_full_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Apr  1 04:51:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.7 MB/s eta 0:00:00


In [ ]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
!mkdir -p {HOME}/data

!wget -q https://media.roboflow.com/notebooks/examples/dog.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-2.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-3.jpeg -P {HOME}/data
!wget -q https://media.roboflow.com/notebooks/examples/dog-4.jpeg -P {HOME}/data

In [ ]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)
mask_predictor = SamPredictor(sam)

In [ ]:
import numpy as np
import cv2
import numpy as np
import supervision as sv
from PIL import Image
import glob
import os

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
#sam = sam_model_registry["<model_type>"](checkpoint="<path/to/checkpoint>")
mask_generator = SamAutomaticMaskGenerator(sam)
image_path='/content/drive/MyDrive/work3/FloodNet/image_extended2/6279.jpg'
image_bgr = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
torch.cuda.empty_cache()
masks = mask_generator.generate(image_rgb)

In [ ]:
import supervision as sv

#mask_annotator = sv.MaskAnnotator(color_map = "index")
detections = sv.Detections.from_sam(masks)
#annotated_image = mask_annotator.annotate(image_bgr, detections)

In [ ]:

image_path='/content/drive/MyDrive/work3/FloodNet/image_extended2'
predict_path='/content/drive/MyDrive/work3/FloodNet/image_extended2_SAM'
images=glob.glob(image_path+'/*.jpg')

for img in images:

  image_bgr = cv2.imread(img)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  height, width, _ = image_rgb.shape
  mask_predictor.set_image(image_rgb)
  basenamef=os.path.basename(img)
  basename=basenamef.split('.')[0]
  #label=label_path+'/'+basename+'.txt'


  lines=[]
  masks_all=[]
  scores_all=[]
  logits_all=[]

  masks, scores, logits = mask_predictor.predict(multimask_output=True)

  masks_all.append(masks)
  scores_all.append(scores)
  logits_all.append(logits)

  if len(masks_all)>0:
    combined_mask=np.zeros_like(masks_all[0])
    for i in range(len(masks_all)):
      combined_mask+=masks_all[i]
    mask_int = np.where(combined_mask, 255, 0).astype(np.uint8)



    full_mask=np.zeros_like((1080,1920))
    for channel in range(mask_int.shape[0]):
      # Extract each channel as a 2D array
      channel_data = mask_int[channel]

      # Create an image from the channel data
      channel_image = Image.fromarray(channel_data, mode='L')  # 'L' mode for grayscale

      # Define the file path to save the channel image
      file_path = f"/content/drive/MyDrive/work3/FloodNet/masks_building/channel/{basename}_mask_{channel}.png"  # Update the file path and format as needed

      # Save the channel image
      #channel_image.save(file_path)
      if channel==0:
        mask1=channel_data
      if channel==1:
        mask2=channel_data
      if channel==2:
        mask3=channel_data
    full_mask=np.zeros_like(mask1)
    combined_mask = np.logical_or(np.logical_or(mask1 == 255, mask2 == 255), mask3 == 255)

  # Assign 255 where the combined condition is True in 'full_mask'
    full_mask[combined_mask] = 255
    file_path = f"/content/drive/MyDrive/work3/FloodNet/image_extended2_SAM/{basename}.jpg"
    cv2.imwrite(file_path,full_mask)



KeyboardInterrupt: 

In [ ]:
import cv2
from segment_anything import SamAutomaticMaskGenerator
import glob
import os
import numpy as np


mask_generator = SamAutomaticMaskGenerator(sam)

image_path='/content/drive/MyDrive/work3/FloodNet/image_extended2'
predict_path='/content/drive/MyDrive/work3/FloodNet/image_extended2_SAM'
images=glob.glob(image_path+'/*.jpg')

for img in images:

  image_bgr = cv2.imread(img)
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  height, width, _ = image_rgb.shape

  result = mask_generator.generate(image_rgb)
  mask, _, _ = result
  mask_int = np.where(mask, 255, 0).astype(np.uint8)
  basenamef=os.path.basename(img)
  basename=basenamef.split('.')[0]
  file_path = f"/content/drive/MyDrive/work3/FloodNet/image_extended2_SAM/{basename}.jpg"
  cv2.imwrite(file_path,mask_int)

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.77 GiB. GPU 0 has a total capacity of 15.77 GiB of which 4.21 GiB is free. Process 5920 has 11.56 GiB memory in use. Of the allocated memory 11.11 GiB is allocated by PyTorch, and 82.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)